## Mercado Libre Tech Challenge Part 1: Exploratory Data Analysis of Lightning Deals

### Objetive: learn outcomes from a lightning deals example dataset.

##### Some guiding qestions
- Are lightning deals useful? Not sufficient data, need sales not in lightning deal regime (put in conclusions).
- What is distribution of deal duration?
- What is the distribution of sales among weekdays and day hours
- What is the distribution of sales among product categories?
- Is there an effect of shipping payment type?

### Imports & Utils

In [150]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import plotly.express as px


def get_dataset_general_features(df: pd.DataFrame) -> None:
    """Prints general info about the given dataset

    Args:
        df (pd.DataFrame): dataset to get general info
    """
    nrows, ncols = df.shape
    print(f"Dataset number of rows: {nrows}")
    print(f"Dataset number of columns: {ncols}\n")
    print("Dataset info:")
    print(f"{df.info()}\n")

    print("Dataset sample:")
    display(df.sample(10, random_state=152))
    

### Load data

In [3]:
data_path = "../data"
data_file_name = "ofertas_relampago.csv"
data = pd.read_csv(f"{data_path}/{data_file_name}")

### Dataset formatting and clean up

##### Original dataset general features

In [4]:
get_dataset_general_features(df=data)

Dataset number of rows: 48746
Dataset number of columns: 13

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48746 entries, 0 to 48745
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   OFFER_START_DATE           48746 non-null  object 
 1   OFFER_START_DTTM           48746 non-null  object 
 2   OFFER_FINISH_DTTM          48746 non-null  object 
 3   OFFER_TYPE                 48746 non-null  object 
 4   INVOLVED_STOCK             48746 non-null  int64  
 5   REMAINING_STOCK_AFTER_END  48746 non-null  int64  
 6   SOLD_AMOUNT                24579 non-null  float64
 7   SOLD_QUANTITY              24579 non-null  float64
 8   ORIGIN                     11316 non-null  object 
 9   SHIPPING_PAYMENT_TYPE      48746 non-null  object 
 10  DOM_DOMAIN_AGG1            48746 non-null  object 
 11  VERTICAL                   48746 non-null  object 
 12  DOMAIN_ID                  

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
45248,2021-07-26,2021-07-26 13:00:00+00:00,2021-07-26 19:00:06+00:00,lightning_deal,15,15,NaN,NaN,NaN,free_shipping,APPAREL ACCESORIES,APP & SPORTS,MLM-RINGS
40833,2021-06-15,2021-06-15 14:00:00+00:00,2021-06-15 16:08:22+00:00,lightning_deal,14,0,269.60,15.0,A,free_shipping,PHARMACEUTICS,BEAUTY & HEALTH,MLM-SURGICAL_AND_INDUSTRIAL_MASKS
17022,2021-07-24,2021-07-24 13:00:00+00:00,2021-07-24 19:00:04+00:00,lightning_deal,5,4,4.26,1.0,NaN,none,PERSONAL CARE,BEAUTY & HEALTH,MLM-NAIL_POLISH
5980,2021-07-13,2021-07-13 19:00:00+00:00,2021-07-14 01:00:00+00:00,lightning_deal,15,15,NaN,NaN,NaN,free_shipping,APPAREL,APP & SPORTS,MLM-SOCKS
6587,2021-06-16,2021-06-16 07:00:00+00:00,2021-06-16 13:00:03+00:00,lightning_deal,5,4,1.73,1.0,NaN,none,PHARMACEUTICS,BEAUTY & HEALTH,MLM-ESSENTIAL_OILS
37310,2021-07-22,2021-07-22 19:00:00+00:00,2021-07-23 01:00:01+00:00,lightning_deal,5,5,NaN,NaN,NaN,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-SOAP_AND_DETERGENT_DISPENSERS
4618,2021-07-23,2021-07-23 07:00:00+00:00,2021-07-23 07:00:00+00:00,lightning_deal,6,6,NaN,NaN,NaN,none,APPAREL ACCESORIES,APP & SPORTS,MLM-WALLETS
45186,2021-07-26,2021-07-26 19:00:00+00:00,2021-07-27 01:00:06+00:00,lightning_deal,5,4,9.91,1.0,NaN,free_shipping,FOODS,CPG,MLM-MILK
31581,2021-07-25,2021-07-25 18:00:00+00:00,2021-07-26 02:00:00+00:00,lightning_deal,10,6,167.63,4.0,A,free_shipping,ELECTRONICS,CE,MLM-WATER_HEATERS
17837,2021-07-10,2021-07-10 13:00:00+00:00,2021-07-10 19:00:03+00:00,lightning_deal,5,5,7.47,1.0,NaN,free_shipping,STATIONARY,HOME & INDUSTRY,MLM-ADHESIVE_TAPES


##### Observations 1
- The dataset contains 48746 rows and 13 columns
- Columns are:
    - `OFFER_START_DATE`: lightning deal start date
    - `OFFER_START_DTTM`: lightning deal start datetime
    - `OFFER_FINISH_DTTM`: lightning deal end datetime  
    - `OFFER_TYPE`: tag describing the deal (lightning_deal)
    - `INVOLVED_STOCK`: number of stock units accesible to the lightning deal
    - `REMAINING_STOCK_AFTER_END`: number of stock units remaining after the deal ended
    - `SOLD_AMOUNT`: percentage of total inventory sold in lighn (?)
    - `SOLD_QUANTITY`: number of units sold
    - `ORIGIN`: seller (?)
    - `SHIPPING_PAYMENT_TYPE`: shipping payment type
    - `VERTICAL`: product categories level 1
    - `DOM_DOMAIN_AGG1`: product categories level 2
    - `DOMAIN_ID`: product categories level 3
    


- Columns with NaNs:
    - `SOLD_AMOUNT`: 24167 rows
    - `SOLD_QUANTITY`: 24167 rows
    - `ORIGIN`: 37430 rows


In [5]:
# Set some columns lists to facilitate data handling
inventory_cols = ["INVOLVED_STOCK", "REMAINING_STOCK_AFTER_END"]
sales_cols = ["SOLD_AMOUNT", "SOLD_QUANTITY"]
category_cols = ["VERTICAL", "DOM_DOMAIN_AGG1", "DOMAIN_ID"]

In [6]:
# [Optional] Show product categories in dataset. Warning: if show_categories is True over 1200 rows are shown.
show_categories = False
if show_categories:
    with pd.option_context("display.max_rows", 1300):
        display(data[["OFFER_TYPE"] + category_cols].groupby(by=category_cols).count().reset_index().rename(columns={"OFFER_TYPE": "count"}))

In [7]:
#Show product origin in dataset
display(data[["OFFER_TYPE", "ORIGIN"]].groupby(by="ORIGIN").count().reset_index().rename(columns={"OFFER_TYPE": "count"}))
        
#Show product shipping payment types in dataset
display(data[["OFFER_TYPE", "SHIPPING_PAYMENT_TYPE"] ].groupby(by="SHIPPING_PAYMENT_TYPE").count().reset_index().rename(columns={"OFFER_TYPE": "count"}))

,ORIGIN,count
0,A,11316


,SHIPPING_PAYMENT_TYPE,count
0,free_shipping,26658
1,none,22088


##### Observations 2

- The ORIGIN column only has one value: `A` (11316 rows)
- The `SHIPPING_PAYMENT_TYPE` column has has only one value: `free_shipping` (26658 rows, while 22088 are `none`)

##### Questions:
- When `SHIPPING_PAYMENT_TYPE` is `none`, does it mean that shipping costs apply? If that is the case, then those products should show a tendency to have less sales compared to products with `free_shipping`.


##### Formatting
- Several columns have `object` dtype which is non convenient for dataframe manipulations efficiency (searches, filtering, grouping), thus, proper formatting will be applied.
- Column `SHIPPING_PAYMENT_TYPE` contains `none` strng values. To normalize this, `none` values will be set to NaN (consistent with column `ORIGIN`).

In [8]:
data = data.astype(
    {
        'OFFER_START_DATE': np.datetime64, 
        'OFFER_START_DTTM': np.datetime64, 
        'OFFER_FINISH_DTTM': np.datetime64,
        'INVOLVED_STOCK': np.int64, 
        'REMAINING_STOCK_AFTER_END': np.int64,
        'SOLD_AMOUNT': np.float64, 
        'SOLD_QUANTITY': np.float64, 
    }
)
data.loc[data["ORIGIN"].isnull(), "ORIGIN"] = None
data.loc[data["SHIPPING_PAYMENT_TYPE"] == "none", "SHIPPING_PAYMENT_TYPE"] = None

# Check formatting output and example
print("Dataset info:")
print(f"{data.info()}\n")

print("Dataset sample:")
display(data.sample(10, random_state=152))

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48746 entries, 0 to 48745
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   OFFER_START_DATE           48746 non-null  datetime64[ns]
 1   OFFER_START_DTTM           48746 non-null  datetime64[ns]
 2   OFFER_FINISH_DTTM          48746 non-null  datetime64[ns]
 3   OFFER_TYPE                 48746 non-null  object        
 4   INVOLVED_STOCK             48746 non-null  int64         
 5   REMAINING_STOCK_AFTER_END  48746 non-null  int64         
 6   SOLD_AMOUNT                24579 non-null  float64       
 7   SOLD_QUANTITY              24579 non-null  float64       
 8   ORIGIN                     11316 non-null  object        
 9   SHIPPING_PAYMENT_TYPE      26658 non-null  object        
 10  DOM_DOMAIN_AGG1            48746 non-null  object        
 11  VERTICAL                   48746 non-null  object    

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
45248,2021-07-26,2021-07-26 13:00:00,2021-07-26 19:00:06,lightning_deal,15,15,NaN,NaN,None,free_shipping,APPAREL ACCESORIES,APP & SPORTS,MLM-RINGS
40833,2021-06-15,2021-06-15 14:00:00,2021-06-15 16:08:22,lightning_deal,14,0,269.60,15.0,A,free_shipping,PHARMACEUTICS,BEAUTY & HEALTH,MLM-SURGICAL_AND_INDUSTRIAL_MASKS
17022,2021-07-24,2021-07-24 13:00:00,2021-07-24 19:00:04,lightning_deal,5,4,4.26,1.0,None,None,PERSONAL CARE,BEAUTY & HEALTH,MLM-NAIL_POLISH
5980,2021-07-13,2021-07-13 19:00:00,2021-07-14 01:00:00,lightning_deal,15,15,NaN,NaN,None,free_shipping,APPAREL,APP & SPORTS,MLM-SOCKS
6587,2021-06-16,2021-06-16 07:00:00,2021-06-16 13:00:03,lightning_deal,5,4,1.73,1.0,None,None,PHARMACEUTICS,BEAUTY & HEALTH,MLM-ESSENTIAL_OILS
37310,2021-07-22,2021-07-22 19:00:00,2021-07-23 01:00:01,lightning_deal,5,5,NaN,NaN,None,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-SOAP_AND_DETERGENT_DISPENSERS
4618,2021-07-23,2021-07-23 07:00:00,2021-07-23 07:00:00,lightning_deal,6,6,NaN,NaN,None,None,APPAREL ACCESORIES,APP & SPORTS,MLM-WALLETS
45186,2021-07-26,2021-07-26 19:00:00,2021-07-27 01:00:06,lightning_deal,5,4,9.91,1.0,None,free_shipping,FOODS,CPG,MLM-MILK
31581,2021-07-25,2021-07-25 18:00:00,2021-07-26 02:00:00,lightning_deal,10,6,167.63,4.0,A,free_shipping,ELECTRONICS,CE,MLM-WATER_HEATERS
17837,2021-07-10,2021-07-10 13:00:00,2021-07-10 19:00:03,lightning_deal,5,5,7.47,1.0,None,free_shipping,STATIONARY,HOME & INDUSTRY,MLM-ADHESIVE_TAPES


##### Clean up
- Dealing with NaNs.
    - Columns `SOLD_AMOUNT` and `SOLD_QUANTITY` have NaNs possibly indicating that no sales occurred in those cases. This can be confirmed by checking if in those rows `INVOLVED_STOCK` == `REMAINING_STOCK_AFTER_END`. If that is the case, then `SOLD_AMOUNT` and `SOLD_QUANTITY` can be set to zero.
    - Column `ORIGIN` has NaNs and, as per Observations 2, it only consists of the value `A`, thus, this column is not much useful at this stage of EDA and the reason for lack of data and potential impact need be discussed with the client.
- Consistency check.
    - `SOLD_QUANTITY` should coincide with the difference between `INVOLVED_STOCK` and `REMAINING_STOCK_AFTER_END`.
    - If `REMAINING_STOCK_AFTER_END` is lower than zero, it means more units than those allowed for lightning deals were sold. 
    
    **Question to the client**: what's the business procedure here? the extra unit sales are cancelled (i.e. `SOLD_QUANTITY` should be made equal to `INVOLVED_STOCK`)?, or the extra unit sales are allowed (i.e. `INVOLVED_STOCK` should be made equal to `SOLD_QUANTITY`)? 

In [9]:
data.loc[data["INVOLVED_STOCK"] == data["REMAINING_STOCK_AFTER_END"]][["SOLD_AMOUNT", "SOLD_QUANTITY"]].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23792 entries, 1 to 48745
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SOLD_AMOUNT    213 non-null    float64
 1   SOLD_QUANTITY  213 non-null    float64
dtypes: float64(2)
memory usage: 557.6 KB


In [10]:
# Display inconsistent sales rows
data.loc[
    (data["INVOLVED_STOCK"] == data["REMAINING_STOCK_AFTER_END"]) 
    & ~(data["SOLD_AMOUNT"].isnull())
    & ~(data["SOLD_QUANTITY"].isnull())
    ]

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
394,2021-06-22,2021-06-22 18:00:00,2021-06-23 01:00:01,lightning_deal,40,40,28.46,3.0,A,free_shipping,PERSONAL CARE,BEAUTY & HEALTH,MLM-FACIAL_SKIN_CARE_PRODUCTS
544,2021-06-22,2021-06-22 14:00:00,2021-06-22 22:00:00,lightning_deal,30,30,4.22,1.0,A,None,TOOLS AND CONSTRUCTION,HOME & INDUSTRY,MLM-TOOL_AND_CONSTRUCTION_SUPPLIES
1033,2021-07-08,2021-07-08 07:00:00,2021-07-08 13:00:05,lightning_deal,15,15,5.40,1.0,None,None,ELECTRONICS,CE,MLM-GAME_CONSOLES
1733,2021-07-08,2021-07-08 07:00:00,2021-07-08 13:00:01,lightning_deal,5,5,5.23,1.0,None,free_shipping,ELECTRONICS,CE,MLM-GAME_CONSOLES_VIDEO_GAMES_AND_ARCADE_MACHINES
1860,2021-07-08,2021-07-08 19:00:00,2021-07-09 01:00:01,lightning_deal,15,15,6.03,1.0,None,None,APPAREL,APP & SPORTS,MLM-PAJAMAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47137,2021-07-06,2021-07-06 13:00:00,2021-07-06 19:00:04,lightning_deal,10,10,4.55,1.0,None,free_shipping,SPORTS,APP & SPORTS,MLM-FOOTBALL_SHIRTS
47167,2021-07-06,2021-07-06 19:00:00,2021-07-07 01:00:05,lightning_deal,15,15,1.33,1.0,None,None,FOODS,CPG,MLM-CHOCOLATES
47189,2021-07-06,2021-07-06 07:00:00,2021-07-06 13:00:01,lightning_deal,15,15,2.53,1.0,None,None,APPAREL ACCESORIES,APP & SPORTS,MLM-NECKLACES
48570,2021-06-19,2021-06-19 19:00:00,2021-06-20 01:00:05,lightning_deal,5,5,5.21,4.0,None,None,SPORTS,APP & SPORTS,MLM-KINESIOLOGY_TAPES


##### Observations 3
213 out of 23792 rows where `INVOLVED_STOCK` == `REMAINING_STOCK_AFTER_END` DO NOT have NaNs in `SOLD_AMOUNT` and `SOLD_QUANTITY`. This appears to be i) an error in the data in terms of not updating `REMAINING_STOCK_AFTER_END` with actual sales, or ii) an error in the reported values in `SOLD_AMOUNT` and `SOLD_QUANTITY`. This needs to be asked to the client. Considering this represents ~0.4% of the data, these rows will NOT be considered in the current analysis, until confirmation of from the client on the source of this inconsistency.

In [11]:
# Drop rows with inconsistent sales.
data.drop(index=data[(data["INVOLVED_STOCK"] == data["REMAINING_STOCK_AFTER_END"]) & ~(data["SOLD_AMOUNT"].isnull()) & ~(data["SOLD_QUANTITY"].isnull())].index, inplace=True)
# Set sales to zero when no change in stock is observed
data.loc[data["INVOLVED_STOCK"] == data["REMAINING_STOCK_AFTER_END"], ("SOLD_AMOUNT", "SOLD_QUANTITY")] = (0, 0)


In [12]:
# Check negative REMAINING_STOCK_AFTER_END
data.loc[data["REMAINING_STOCK_AFTER_END"] < 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1273 entries, 0 to 48719
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   OFFER_START_DATE           1273 non-null   datetime64[ns]
 1   OFFER_START_DTTM           1273 non-null   datetime64[ns]
 2   OFFER_FINISH_DTTM          1273 non-null   datetime64[ns]
 3   OFFER_TYPE                 1273 non-null   object        
 4   INVOLVED_STOCK             1273 non-null   int64         
 5   REMAINING_STOCK_AFTER_END  1273 non-null   int64         
 6   SOLD_AMOUNT                1241 non-null   float64       
 7   SOLD_QUANTITY              1241 non-null   float64       
 8   ORIGIN                     167 non-null    object        
 9   SHIPPING_PAYMENT_TYPE      184 non-null    object        
 10  DOM_DOMAIN_AGG1            1273 non-null   object        
 11  VERTICAL                   1273 non-null   object        
 12  DOMAI

In [13]:
# Show inconsisten sales rows
data.loc[(data["REMAINING_STOCK_AFTER_END"] < 0) & (data["SOLD_AMOUNT"].isnull()) & (data["SOLD_QUANTITY"].isnull())]

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
8,2021-06-22,2021-06-22 19:00:00,2021-06-22 23:05:32,lightning_deal,10,-1,NaN,NaN,None,free_shipping,COMPUTERS,CE,MLM-HEADPHONES
3399,2021-06-26,2021-06-26 13:00:00,2021-06-26 15:23:17,lightning_deal,5,-1,NaN,NaN,None,None,HOME&DECOR,HOME & INDUSTRY,MLM-DRINKING_GLASSES
4210,2021-07-23,2021-07-23 07:00:00,2021-07-23 10:24:24,lightning_deal,15,-2,NaN,NaN,None,None,COMPUTERS,CE,MLM-SPEAKERS
4219,2021-07-23,2021-07-23 07:00:00,2021-07-23 10:24:24,lightning_deal,15,-2,NaN,NaN,None,None,COMPUTERS,CE,MLM-SPEAKERS
5967,2021-07-13,2021-07-13 19:00:00,2021-07-13 23:57:00,lightning_deal,5,-3,NaN,NaN,None,None,ELECTRONICS,CE,MLM-GAMEPADS_AND_JOYSTICKS
6652,2021-06-16,2021-06-16 07:00:00,2021-06-16 10:19:57,lightning_deal,15,-1,NaN,NaN,None,None,COMPUTERS,CE,MLM-SPEAKERS
6653,2021-06-16,2021-06-16 07:00:00,2021-06-16 10:19:56,lightning_deal,15,-1,NaN,NaN,None,None,COMPUTERS,CE,MLM-SPEAKERS
6656,2021-06-16,2021-06-16 07:00:00,2021-06-16 10:19:56,lightning_deal,15,-1,NaN,NaN,None,None,COMPUTERS,CE,MLM-SPEAKERS
6657,2021-06-16,2021-06-16 07:00:00,2021-06-16 10:19:57,lightning_deal,15,-1,NaN,NaN,None,None,COMPUTERS,CE,MLM-SPEAKERS
6658,2021-06-16,2021-06-16 07:00:00,2021-06-16 10:19:57,lightning_deal,15,-1,NaN,NaN,None,None,COMPUTERS,CE,MLM-SPEAKERS


##### Observations 4
Two types of rows with `REMAINING_STOCK_AFTER_END` < 0 (~2.6% of total rows):
- Consistent: `SOLD_QUANTITY` +  `REMAINING_STOCK_AFTER_END` = `INVOLVED_STOCK`
- Inconsistent: `SOLD_QUANTITY` and `SOLD_AMOUNT` are NaN (~0.07% of total rows)

The inconsistent cases seem to be an i) an error in the values of `REMAINING_STOCK_AFTER_END` (no actual sales occurred) or ii) `SOLD_QUANTITY` and `SOLD_AMOUNT` not being updated. This should be checked with the client. 

Considering that rows with negative `REMAINING_STOCK_AFTER_END` reprensent ~2.6% of total rows in the dataframe, they will be dropped, until confirmation from the client on the reason for these data issue.

In [14]:
# Drop inconsistent sales rows
# data.drop(index=data.loc[(data["REMAINING_STOCK_AFTER_END"] < 0) & (data["SOLD_AMOUNT"].isnull()) & (data["SOLD_QUANTITY"].isnull())].index, inplace=True)
data.drop(index=data.loc[(data["REMAINING_STOCK_AFTER_END"] < 0)].index, inplace=True)

##### Check current dataset features

In [15]:
get_dataset_general_features(df=data)

Dataset number of rows: 47260
Dataset number of columns: 13

Dataset info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 47260 entries, 1 to 48745
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   OFFER_START_DATE           47260 non-null  datetime64[ns]
 1   OFFER_START_DTTM           47260 non-null  datetime64[ns]
 2   OFFER_FINISH_DTTM          47260 non-null  datetime64[ns]
 3   OFFER_TYPE                 47260 non-null  object        
 4   INVOLVED_STOCK             47260 non-null  int64         
 5   REMAINING_STOCK_AFTER_END  47260 non-null  int64         
 6   SOLD_AMOUNT                46704 non-null  float64       
 7   SOLD_QUANTITY              46704 non-null  float64       
 8   ORIGIN                     11121 non-null  object        
 9   SHIPPING_PAYMENT_TYPE      26369 non-null  object        
 10  DOM_DOMAIN_AGG1            47260 non-null  object      

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
18929,2021-07-04,2021-07-04 07:00:00,2021-07-04 13:00:03,lightning_deal,15,15,0.00,0.0,None,None,APPAREL ACCESORIES,APP & SPORTS,MLM-FANNY_PACKS
14893,2021-07-09,2021-07-09 13:00:00,2021-07-09 19:00:05,lightning_deal,15,2,41.03,13.0,None,free_shipping,SPORTS,APP & SPORTS,MLM-RESISTANCE_BANDS
30898,2021-07-11,2021-07-11 13:00:00,2021-07-11 19:00:02,lightning_deal,15,15,0.00,0.0,None,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-OUTDOOR_TABLES
47261,2021-07-06,2021-07-06 13:00:00,2021-07-06 19:00:00,lightning_deal,15,12,5.35,3.0,None,None,STATIONARY,HOME & INDUSTRY,MLM-MARKERS_AND_HIGHLIGHTERS
15916,2021-06-23,2021-06-23 19:00:00,2021-06-24 01:00:00,lightning_deal,5,5,0.00,0.0,None,None,"BOOKS, MULTIMEDIA & OTHER E!",ENTERTAINMENT,MLM-MUSIC_ALBUMS
3663,2021-06-26,2021-06-26 07:00:00,2021-06-26 13:00:00,lightning_deal,15,15,0.00,0.0,None,None,COMPUTERS,CE,MLM-DATA_CABLES_AND_ADAPTERS
26318,2021-06-17,2021-06-17 13:00:00,2021-06-17 13:00:04,lightning_deal,15,15,0.00,0.0,None,free_shipping,FOOTWEAR,APP & SPORTS,MLM-BOOTS_AND_BOOTIES
44099,2021-07-12,2021-07-12 07:00:00,2021-07-12 13:00:02,lightning_deal,10,10,0.00,0.0,None,free_shipping,COMPUTERS,CE,MLM-MICROPHONES
13626,2021-07-19,2021-07-19 13:00:00,2021-07-19 13:00:00,lightning_deal,10,10,0.00,0.0,None,free_shipping,AUTOPARTS,ACC,MLM-VEHICLE_ACCESSORIES
37828,2021-07-22,2021-07-22 13:00:00,2021-07-22 19:00:01,lightning_deal,15,9,6.38,6.0,None,None,TOYS AND GAMES,T & B,MLM-ACTION_FIGURES


In [16]:
data.loc[data["SOLD_AMOUNT"].isnull()]

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
299,2021-06-22,2021-06-22 13:00:00,2021-06-22 19:00:01,lightning_deal,15,7,NaN,NaN,None,free_shipping,ELECTRONICS,CE,MLM-FANS
522,2021-06-22,2021-06-22 13:00:00,2021-06-22 21:00:00,lightning_deal,100,96,NaN,NaN,A,free_shipping,MOBILE,CE,MLM-TABLETS
623,2021-06-22,2021-06-22 07:00:00,2021-06-22 13:00:04,lightning_deal,10,5,NaN,NaN,None,None,COMPUTERS,CE,MLM-MICROPHONES
641,2021-06-22,2021-06-22 19:00:00,2021-06-22 21:22:13,lightning_deal,15,0,NaN,NaN,None,free_shipping,ELECTRONICS,CE,MLM-MEMORY_CARDS
668,2021-06-22,2021-06-22 13:00:00,2021-06-22 19:00:02,lightning_deal,5,3,NaN,NaN,None,free_shipping,INDUSTRY,HOME & INDUSTRY,MLM-OFFICE_CHAIRS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48498,2021-06-19,2021-06-19 13:00:00,2021-06-19 21:00:00,lightning_deal,100,91,NaN,NaN,A,free_shipping,MOBILE,CE,MLM-TABLETS
48586,2021-06-19,2021-06-19 14:00:00,2021-06-19 22:00:00,lightning_deal,30,29,NaN,NaN,A,free_shipping,MOBILE,CE,MLM-CELLPHONES
48610,2021-06-19,2021-06-19 15:00:00,2021-06-19 23:00:00,lightning_deal,25,22,NaN,NaN,A,free_shipping,TOOLS AND CONSTRUCTION,HOME & INDUSTRY,MLM-POWER_GRINDERS
48660,2021-06-19,2021-06-19 15:00:00,2021-06-19 23:00:01,lightning_deal,10,9,NaN,NaN,A,free_shipping,ELECTRONICS,CE,MLM-HAIR_CLIPPERS


##### Observation 5
556 rows contain NaNs in `SOLD_AMOUNT` and `SOLD_QUANTITY` while `INVOLVED_STOCK` is different than `REMAINING_STOCK_AFTER_END`, thus, i) there is an error in the `REMAINING_STOCK_AFTER_END`, or `SOLD_AMOUNT` and `SOLD_QUANTITY` were not updated. This issue occurs in ~1.1% of total rows and will be dropped untils discussing this issue with the client.

In [17]:
data.drop(index=data[(data["INVOLVED_STOCK"] > data["REMAINING_STOCK_AFTER_END"]) & (data["SOLD_AMOUNT"].isnull()) & (data["SOLD_QUANTITY"].isnull())].index, inplace=True)

##### Dataset features after formatting and clean up.

In [18]:
get_dataset_general_features(df=data)

Dataset number of rows: 46704
Dataset number of columns: 13

Dataset info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 46704 entries, 1 to 48745
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   OFFER_START_DATE           46704 non-null  datetime64[ns]
 1   OFFER_START_DTTM           46704 non-null  datetime64[ns]
 2   OFFER_FINISH_DTTM          46704 non-null  datetime64[ns]
 3   OFFER_TYPE                 46704 non-null  object        
 4   INVOLVED_STOCK             46704 non-null  int64         
 5   REMAINING_STOCK_AFTER_END  46704 non-null  int64         
 6   SOLD_AMOUNT                46704 non-null  float64       
 7   SOLD_QUANTITY              46704 non-null  float64       
 8   ORIGIN                     10888 non-null  object        
 9   SHIPPING_PAYMENT_TYPE      25908 non-null  object        
 10  DOM_DOMAIN_AGG1            46704 non-null  object      

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
11665,2021-07-27,2021-07-27 13:00:00,2021-07-27 13:00:02,lightning_deal,15,15,0.00,0.0,None,free_shipping,ELECTRONICS,CE,MLM-GAME_CONSOLES_VIDEO_GAMES_AND_ARCADE_MACHINES
36466,2021-06-11,2021-06-11 13:00:00,2021-06-11 19:00:02,lightning_deal,15,13,5.53,2.0,None,None,PARTY,T & B,MLM-BALLOONS
18328,2021-07-10,2021-07-10 13:00:00,2021-07-10 13:00:01,lightning_deal,5,5,0.00,0.0,None,free_shipping,MOBILE,CE,MLM-MOBILE_DEVICE_CHARGERS
37456,2021-07-22,2021-07-22 13:00:00,2021-07-22 19:00:01,lightning_deal,15,12,4.18,3.0,None,None,APPAREL ACCESORIES,APP & SPORTS,MLM-RINGS
13271,2021-07-19,2021-07-19 19:00:00,2021-07-20 01:00:01,lightning_deal,15,6,25.43,9.0,None,None,SPORTS,APP & SPORTS,MLM-THERMOSES
11251,2021-07-27,2021-07-27 18:00:00,2021-07-28 01:00:01,lightning_deal,6,6,0.00,0.0,A,None,PERSONAL CARE,BEAUTY & HEALTH,MLM-BEAUTY_AND_PERSONAL_CARE_SUPPLIES
42044,2021-06-05,2021-06-05 07:00:00,2021-06-05 13:00:02,lightning_deal,15,15,0.00,0.0,None,None,PERSONAL CARE,BEAUTY & HEALTH,MLM-NAIL_POLISH
23907,2021-07-31,2021-07-31 19:00:00,2021-08-01 01:00:01,lightning_deal,5,3,2.54,2.0,None,None,COMPUTERS,CE,MLM-NETWORK_CABLES
46573,2021-07-16,2021-07-16 14:00:00,2021-07-16 22:00:01,lightning_deal,20,18,27.12,2.0,A,free_shipping,ELECTRONICS,CE,MLM-PORTABLE_DVD_PLAYERS
36061,2021-06-11,2021-06-11 13:00:00,2021-06-11 19:00:01,lightning_deal,15,13,11.11,2.0,None,free_shipping,APPAREL,APP & SPORTS,MLM-BLOUSES


### Dataset Analysis

#### Deal durations

In [19]:
data["OFFER_DURATION_MINUTES"] = (data["OFFER_FINISH_DTTM"] - data["OFFER_START_DTTM"]).astype('timedelta64[m]')
data["OFFER_DURATION_HOURS"] = (data["OFFER_FINISH_DTTM"] - data["OFFER_START_DTTM"]).astype('timedelta64[h]')
data["OFFER_START_WEEKDAY"] = data["OFFER_START_DTTM"].dt.weekday
data["OFFER_START_MONTH"] = data["OFFER_START_DTTM"].dt.month
data["OFFER_START_DAY"] = data["OFFER_START_DTTM"].dt.day
data["OFFER_START_DAYNAME"] = data["OFFER_START_DTTM"].dt.day_name()
data["OFFER_START_HOUR"] = data["OFFER_START_DTTM"].dt.hour
data["OFFER_START_YEAR"] = data["OFFER_START_DTTM"].dt.year

In [20]:
display(data.sample(10, random_state=152))

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,...,VERTICAL,DOMAIN_ID,OFFER_DURATION_MINUTES,OFFER_DURATION_HOURS,OFFER_START_WEEKDAY,OFFER_START_MONTH,OFFER_START_DAY,OFFER_START_DAYNAME,OFFER_START_HOUR,OFFER_START_YEAR
11665,2021-07-27,2021-07-27 13:00:00,2021-07-27 13:00:02,lightning_deal,15,15,0.00,0.0,None,free_shipping,...,CE,MLM-GAME_CONSOLES_VIDEO_GAMES_AND_ARCADE_MACHINES,0.0,0.0,1,7,27,Tuesday,13,2021
36466,2021-06-11,2021-06-11 13:00:00,2021-06-11 19:00:02,lightning_deal,15,13,5.53,2.0,None,None,...,T & B,MLM-BALLOONS,360.0,6.0,4,6,11,Friday,13,2021
18328,2021-07-10,2021-07-10 13:00:00,2021-07-10 13:00:01,lightning_deal,5,5,0.00,0.0,None,free_shipping,...,CE,MLM-MOBILE_DEVICE_CHARGERS,0.0,0.0,5,7,10,Saturday,13,2021
37456,2021-07-22,2021-07-22 13:00:00,2021-07-22 19:00:01,lightning_deal,15,12,4.18,3.0,None,None,...,APP & SPORTS,MLM-RINGS,360.0,6.0,3,7,22,Thursday,13,2021
13271,2021-07-19,2021-07-19 19:00:00,2021-07-20 01:00:01,lightning_deal,15,6,25.43,9.0,None,None,...,APP & SPORTS,MLM-THERMOSES,360.0,6.0,0,7,19,Monday,19,2021
11251,2021-07-27,2021-07-27 18:00:00,2021-07-28 01:00:01,lightning_deal,6,6,0.00,0.0,A,None,...,BEAUTY & HEALTH,MLM-BEAUTY_AND_PERSONAL_CARE_SUPPLIES,420.0,7.0,1,7,27,Tuesday,18,2021
42044,2021-06-05,2021-06-05 07:00:00,2021-06-05 13:00:02,lightning_deal,15,15,0.00,0.0,None,None,...,BEAUTY & HEALTH,MLM-NAIL_POLISH,360.0,6.0,5,6,5,Saturday,7,2021
23907,2021-07-31,2021-07-31 19:00:00,2021-08-01 01:00:01,lightning_deal,5,3,2.54,2.0,None,None,...,CE,MLM-NETWORK_CABLES,360.0,6.0,5,7,31,Saturday,19,2021
46573,2021-07-16,2021-07-16 14:00:00,2021-07-16 22:00:01,lightning_deal,20,18,27.12,2.0,A,free_shipping,...,CE,MLM-PORTABLE_DVD_PLAYERS,480.0,8.0,4,7,16,Friday,14,2021
36061,2021-06-11,2021-06-11 13:00:00,2021-06-11 19:00:01,lightning_deal,15,13,11.11,2.0,None,free_shipping,...,APP & SPORTS,MLM-BLOUSES,360.0,6.0,4,6,11,Friday,13,2021


In [21]:
# Get years, months and weekdays of available lightning deals
print(f"Deal start years: {np.sort(data['OFFER_START_YEAR'].unique())}")
print(f"Deal start months: {np.sort(data['OFFER_START_MONTH'].unique())}")
print(f"Deal start weekdays: {np.sort(data['OFFER_START_WEEKDAY'].unique())}")
print(f"Deal start month days: {np.sort(data['OFFER_START_DAY'].unique())}")
print(f"Deal start days: {data['OFFER_START_DAYNAME'].unique()}")
print(f"Deal start hour: {np.sort(data['OFFER_START_HOUR'].unique())}")

Deal start years: [2021]
Deal start months: [6 7]
Deal start weekdays: [0 1 2 3 4 5 6]
Deal start month days: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
Deal start days: ['Tuesday' 'Thursday' 'Wednesday' 'Saturday' 'Friday' 'Sunday' 'Monday']
Deal start hour: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


##### Observation
Deals took place in June and July of 2021, starting dates and times spanning all days of the month and every hour of the day.

In [22]:
# Get distribution of deal durations
print("Duration distribution")
display(
    data[["OFFER_DURATION_HOURS", "OFFER_DURATION_MINUTES"]]
    .groupby(by="OFFER_DURATION_HOURS")
    .agg(
        {
            "OFFER_DURATION_MINUTES": ["count", "min", "mean", "max"]
        }
    )
    .reset_index()
)

Duration distribution


OFFER_DURATION_HOURS OFFER_DURATION_MINUTES                             
                                         count     min         mean     max
0                   0.0                   4000     0.0     1.247500    59.0
1                   1.0                    300    60.0    90.266667   119.0
2                   2.0                    449   120.0   148.848552   179.0
3                   3.0                    509   180.0   210.489194   239.0
4                   4.0                    681   240.0   269.734214   299.0
5                   5.0                   1132   300.0   320.729682   359.0
6                   6.0                  29998   360.0   360.593006   419.0
7                   7.0                   1975   420.0   420.745823   477.0
8                   8.0                   7492   480.0   480.376268   536.0
9                   9.0                      3   541.0   543.000000   546.0
10                 10.0                      4   616.0   626.000000   655.0
11                 11.0                      4   661.0   697.000000   716.0
12                 12.0                    152   721.0   721.486842   722.0
13                 14.0                      3   877.0   878.000000   880.0
14                 15.0                      1   937.0   937.000000   937.0
15                104.0                      1  6285.0  6285.000000  6285.0

In [218]:
fig = px.histogram(data, x="OFFER_DURATION_MINUTES", hover_data=data.columns, width=1200, height=700, barmode="overlay", range_x=(0,722), nbins=int(data["OFFER_DURATION_MINUTES"].max()/10))
fig.show()

##### Observation 6
Lightning deals durantion ranges from 0 minutes to 104 hours (> 4.3 days) with most cases falling in the range 6-8 hours. Durantions

In [24]:
display(data[data["OFFER_DURATION_MINUTES"] == 0])

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,...,VERTICAL,DOMAIN_ID,OFFER_DURATION_MINUTES,OFFER_DURATION_HOURS,OFFER_START_WEEKDAY,OFFER_START_MONTH,OFFER_START_DAY,OFFER_START_DAYNAME,OFFER_START_HOUR,OFFER_START_YEAR
18,2021-06-22,2021-06-22 19:00:00,2021-06-22 19:00:03,lightning_deal,10,10,0.0,0.0,None,free_shipping,...,T & B,MLM-PARTY_SUPPLIES,0.0,0.0,1,6,22,Tuesday,19,2021
25,2021-06-22,2021-06-22 13:00:00,2021-06-22 13:00:00,lightning_deal,5,5,0.0,0.0,None,free_shipping,...,HOME & INDUSTRY,MLM-SHOWER_HEADS,0.0,0.0,1,6,22,Tuesday,13,2021
72,2021-06-22,2021-06-22 07:00:00,2021-06-22 07:00:00,lightning_deal,5,5,0.0,0.0,None,free_shipping,...,ENTERTAINMENT,MLM-BOOKS,0.0,0.0,1,6,22,Tuesday,7,2021
90,2021-06-22,2021-06-22 16:00:00,2021-06-22 16:00:00,lightning_deal,3,3,0.0,0.0,A,free_shipping,...,CPG,MLM-CATS_AND_DOGS_FOODS,0.0,0.0,1,6,22,Tuesday,16,2021
98,2021-06-22,2021-06-22 16:00:00,2021-06-22 16:00:00,lightning_deal,3,3,0.0,0.0,A,free_shipping,...,CPG,MLM-CATS_AND_DOGS_FOODS,0.0,0.0,1,6,22,Tuesday,16,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48646,2021-06-19,2021-06-19 13:00:00,2021-06-19 13:00:00,lightning_deal,8,8,0.0,0.0,None,None,...,APP & SPORTS,MLM-UNDERPANTS,0.0,0.0,5,6,19,Saturday,13,2021
48647,2021-06-19,2021-06-19 19:00:00,2021-06-19 19:00:00,lightning_deal,5,5,0.0,0.0,None,free_shipping,...,APP & SPORTS,MLM-UNDERPANTS,0.0,0.0,5,6,19,Saturday,19,2021
48700,2021-06-19,2021-06-19 07:00:00,2021-06-19 07:00:00,lightning_deal,10,10,0.0,0.0,None,None,...,CE,MLM-PLAGUES_ULTRASONIC_REPELLENTS,0.0,0.0,5,6,19,Saturday,7,2021
48709,2021-06-19,2021-06-19 07:00:00,2021-06-19 07:00:00,lightning_deal,5,5,0.0,0.0,None,None,...,HOME & INDUSTRY,MLM-PILLOWS,0.0,0.0,5,6,19,Saturday,7,2021


##### Observation 7
Around 7% of rows have a deal duration of zero minutes. This needs to be checked with the client, but will be removed for the present analysis. 

In [25]:
data.drop(index=data[data["OFFER_DURATION_MINUTES"] == 0].index, inplace=True)

In [26]:
print("Duration < 15 min")
display(data.loc[
    (data["OFFER_DURATION_MINUTES"] > 0)
    & (data["OFFER_DURATION_MINUTES"] < 15)
    ])

Duration < 15 min


,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,...,VERTICAL,DOMAIN_ID,OFFER_DURATION_MINUTES,OFFER_DURATION_HOURS,OFFER_START_WEEKDAY,OFFER_START_MONTH,OFFER_START_DAY,OFFER_START_DAYNAME,OFFER_START_HOUR,OFFER_START_YEAR
889,2021-07-08,2021-07-08 11:00:00,2021-07-08 11:01:46,lightning_deal,60,60,0.00,0.0,A,None,...,APP & SPORTS,MLM-BLOUSES,1.0,0.0,3,7,8,Thursday,11,2021
979,2021-07-08,2021-07-08 11:00:00,2021-07-08 11:01:46,lightning_deal,21,21,0.00,0.0,A,None,...,APP & SPORTS,MLM-SWEATERS_AND_CARDIGANS,1.0,0.0,3,7,8,Thursday,11,2021
2243,2021-06-02,2021-06-02 07:00:00,2021-06-02 07:01:09,lightning_deal,15,15,0.00,0.0,None,free_shipping,...,APP & SPORTS,MLM-SHIRTS,1.0,0.0,2,6,2,Wednesday,7,2021
2417,2021-06-02,2021-06-02 13:00:00,2021-06-02 13:01:43,lightning_deal,5,5,0.00,0.0,None,free_shipping,...,CE,MLM-COMPUTER_EQUIPMENT_AND_SPARE_PARTS,1.0,0.0,2,6,2,Wednesday,13,2021
2841,2021-06-12,2021-06-12 19:00:00,2021-06-12 19:07:25,lightning_deal,5,0,9.65,5.0,None,None,...,ACC,MLM-AUTOMOTIVE_LED_LIGHT_BARS,7.0,0.0,5,6,12,Saturday,19,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46490,2021-07-16,2021-07-16 16:00:00,2021-07-16 16:01:06,lightning_deal,7,7,0.00,0.0,A,None,...,APP & SPORTS,MLM-SUNGLASSES,1.0,0.0,4,7,16,Friday,16,2021
46494,2021-07-16,2021-07-16 11:00:00,2021-07-16 11:01:50,lightning_deal,44,44,0.00,0.0,A,free_shipping,...,APP & SPORTS,MLM-SUNGLASSES,1.0,0.0,4,7,16,Friday,11,2021
46497,2021-07-16,2021-07-16 11:00:00,2021-07-16 11:01:50,lightning_deal,3,3,0.00,0.0,A,free_shipping,...,APP & SPORTS,MLM-SUNGLASSES,1.0,0.0,4,7,16,Friday,11,2021
46523,2021-07-16,2021-07-16 11:00:00,2021-07-16 11:01:50,lightning_deal,148,148,0.00,0.0,A,free_shipping,...,APP & SPORTS,MLM-PANTS,1.0,0.0,4,7,16,Friday,11,2021


##### Observation 7
What is the minimum deal duration? having a deal for too few minutes doesn't seem to make sense, and entries in the dataset with such low durations could be an error in the data. For the present analysis, durations less than 15 minutes will be removed (~0.1% total rows).

In [50]:
# To compare sales of different lightning deal events, because each event can have a different duration (see analysis above) we need to normalize by a certain time unit. Herein sales per hour will be used.
data["OFFER_DURATION_HOUR_BUCKETS"] = pd.cut(x=data["OFFER_DURATION_MINUTES"], bins = np.array(range(0,106)) * 60, labels=np.array(range(1,106)))
data = data.astype({"OFFER_DURATION_HOUR_BUCKETS": np.int64})
data["SOLD_QUANTITY_HOURLY"] = data["SOLD_QUANTITY"] / data["OFFER_DURATION_HOUR_BUCKETS"]

In [51]:
display(data.sample(10))

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,...,OFFER_DURATION_MINUTES,OFFER_DURATION_HOURS,OFFER_START_WEEKDAY,OFFER_START_MONTH,OFFER_START_DAY,OFFER_START_DAYNAME,OFFER_START_HOUR,OFFER_START_YEAR,OFFER_DURATION_HOUR_BUCKETS,SOLD_QUANTITY_HOURLY
44528,2021-06-29,2021-06-29 07:00:00,2021-06-29 13:00:05,lightning_deal,5,5,0.00,0.0,None,None,...,360.0,6.0,1,6,29,Tuesday,7,2021,6,0.000000
1380,2021-07-08,2021-07-08 17:00:00,2021-07-08 21:08:19,lightning_deal,1,0,4.76,1.0,A,free_shipping,...,248.0,4.0,3,7,8,Thursday,17,2021,5,0.200000
24889,2021-06-20,2021-06-20 15:00:00,2021-06-20 23:00:00,lightning_deal,10,2,2.39,8.0,A,None,...,480.0,8.0,6,6,20,Sunday,15,2021,8,1.000000
36929,2021-06-18,2021-06-18 13:00:00,2021-06-18 19:00:02,lightning_deal,5,3,1.61,2.0,None,None,...,360.0,6.0,4,6,18,Friday,13,2021,6,0.333333
10689,2021-07-17,2021-07-17 15:00:00,2021-07-17 23:00:01,lightning_deal,60,60,0.00,0.0,A,free_shipping,...,480.0,8.0,5,7,17,Saturday,15,2021,8,0.000000
32832,2021-07-15,2021-07-15 07:00:00,2021-07-15 13:00:03,lightning_deal,5,5,0.00,0.0,None,None,...,360.0,6.0,3,7,15,Thursday,7,2021,6,0.000000
498,2021-06-22,2021-06-22 20:00:00,2021-06-22 21:48:16,lightning_deal,15,0,177.71,15.0,A,free_shipping,...,108.0,1.0,1,6,22,Tuesday,20,2021,2,7.500000
45773,2021-07-26,2021-07-26 13:00:00,2021-07-26 14:53:01,lightning_deal,5,0,3.78,5.0,None,None,...,113.0,1.0,0,7,26,Monday,13,2021,2,2.500000
23614,2021-07-31,2021-07-31 12:00:00,2021-07-31 20:00:00,lightning_deal,8,8,0.00,0.0,A,free_shipping,...,480.0,8.0,5,7,31,Saturday,12,2021,8,0.000000
13338,2021-07-19,2021-07-19 13:00:00,2021-07-19 21:00:00,lightning_deal,13,7,92.31,6.0,A,free_shipping,...,480.0,8.0,0,7,19,Monday,13,2021,8,0.750000


In [27]:
data.drop(index=data[data["OFFER_DURATION_MINUTES"] < 15].index, inplace=True)

#### Distribution of sales at different time granularities (monthly, day of the month, day of the week and hour of the day)

In [167]:
current_date = data['OFFER_START_DTTM'].min()
end_date = data['OFFER_FINISH_DTTM'].max()

# # Group by date
data_hourly_list = []
data_dict = data.to_dict(orient="records")
while current_date <= end_date:
    hourly_deal = {
        "datetime": current_date, 
        "hourly_sales": 0,
        "hourly_sales_norm": 0, 
        "num_deals": 0, 
        # "categories": []
        }
    for d in data_dict:
        if d["OFFER_START_DTTM"] <= current_date < d["OFFER_FINISH_DTTM"]:
            hourly_deal["num_deals"] += 1
            hourly_deal["hourly_sales"] = hourly_deal["hourly_sales"] + d["SOLD_QUANTITY"] / d["OFFER_DURATION_HOUR_BUCKETS"]
            # hourly_deal["categories"].append(
            #     {
            #         "level_1": d["VERTICAL"],
            #         "level_2": d["DOM_DOMAIN_AGG1"],
            #         "level_3": d["DOMAIN_ID"]
            #     }
            # )
    if hourly_deal["num_deals"] > 0:
        hourly_deal["hourly_sales_norm"] = hourly_deal["hourly_sales"] / hourly_deal["num_deals"]
    data_hourly_list.append(hourly_deal)
    current_date = current_date + timedelta(hours=1)

data_hourly = pd.DataFrame(data_hourly_list)

# Get years, months and weekdays of available lightning deals
data_hourly["datetime_month"] = data_hourly["datetime"].dt.month
data_hourly["datatime_day"] = data_hourly["datetime"].dt.day
data_hourly["datetime_dayname"] = data_hourly["datetime"].dt.day_name()
data_hourly["datetime_hour"] = data_hourly["datetime"].dt.hour
data_hourly["date"] = data_hourly["datetime"].dt.date
data_hourly["weekday"] = data_hourly["datetime"].dt.weekday

# Keep only hours with deals (actual data)
data_hourly.drop(index=data_hourly[data_hourly["num_deals"] == 0].index, inplace=True)


In [168]:
data_hourly.head(10)

,datetime,hourly_sales,hourly_sales_norm,num_deals,datetime_month,datatime_day,datetime_dayname,datetime_hour,date,weekday
0,2021-06-01 07:00:00,39.821429,0.176984,225,6,1,Tuesday,7,2021-06-01,1
1,2021-06-01 08:00:00,39.821429,0.176984,225,6,1,Tuesday,8,2021-06-01,1
2,2021-06-01 09:00:00,39.821429,0.176984,225,6,1,Tuesday,9,2021-06-01,1
3,2021-06-01 10:00:00,96.854762,0.424802,228,6,1,Tuesday,10,2021-06-01,1
4,2021-06-01 11:00:00,116.771429,0.512155,228,6,1,Tuesday,11,2021-06-01,1
5,2021-06-01 12:00:00,142.821429,0.590171,242,6,1,Tuesday,12,2021-06-01,1
6,2021-06-01 13:00:00,204.958333,0.437945,468,6,1,Tuesday,13,2021-06-01,1
7,2021-06-01 14:00:00,247.303571,0.958541,258,6,1,Tuesday,14,2021-06-01,1
8,2021-06-01 15:00:00,277.828571,1.068571,260,6,1,Tuesday,15,2021-06-01,1
9,2021-06-01 16:00:00,218.778571,0.835033,262,6,1,Tuesday,16,2021-06-01,1


In [169]:
# Sales by month
sales_by_month = data_hourly[["hourly_sales_norm", "datetime_month"]].groupby(by="datetime_month").agg({"hourly_sales_norm": ["mean"]}).reset_index()
sales_by_month.columns = ["_".join(col) for col in sales_by_month.columns.values]
sales_by_month.rename(columns={"datetime_month_": "datetime_month"}, inplace=True)
display(sales_by_month)

,datetime_month,hourly_sales_norm_mean
0,6,0.648555
1,7,0.875374
2,8,0.186502


In [170]:
# Sales by month
data[["OFFER_START_MONTH", "OFFER_TYPE"]].groupby(by="OFFER_START_MONTH").agg("count").reset_index().rename(columns={"OFFER_TYPE": "count"})

,OFFER_START_MONTH,count
0,6,16169
1,7,26651


In [171]:
data_hourly.loc[data_hourly["datetime_month"] == 8]

,datetime,hourly_sales,hourly_sales_norm,num_deals,datetime_month,datatime_day,datetime_dayname,datetime_hour,date,weekday
1457,2021-08-01 00:00:00,85.416667,0.294540,290,8,1,Sunday,0,2021-08-01,6
1458,2021-08-01 01:00:00,64.916667,0.264966,245,8,1,Sunday,1,2021-08-01,6
1459,2021-08-01 02:00:00,0.000000,0.000000,1,8,1,Sunday,2,2021-08-01,6


In [204]:
fig = px.histogram(data_hourly, x="hourly_sales_norm", color="datetime_month", hover_data=data_hourly.columns, width=1200, height=500, barmode="overlay", range_x=(0,4), nbins=20)
fig.show()

In [173]:
# Min/max dates of lightning deals
print(f"First date of lightning deals: {data['OFFER_START_DTTM'].min()}")
print(f"Last date of lightning deals: {data['OFFER_FINISH_DTTM'].max()}\n")
print(f"First date of lightning deals (with sales): {data.loc[data['SOLD_QUANTITY'] > 0]['OFFER_START_DTTM'].min()}")
print(f"Last date of lightning deals (with sales): {data.loc[data['SOLD_QUANTITY'] > 0]['OFFER_FINISH_DTTM'].max()}")


First date of lightning deals: 2021-06-01 07:00:00
Last date of lightning deals: 2021-08-01 03:00:00

First date of lightning deals (with sales): 2021-06-01 07:00:00
Last date of lightning deals (with sales): 2021-08-01 01:00:34


##### Observation 8
- The lighting deals started on 2021-06-01 07:00:00 and finished on 2021-08-01 03:00:00. 
- Sales started right away in the first time period available (starting date and time: 2021-06-01 07:00:00), and lasted almost until the very last available period (ending date and time: 2021-08-01 01:00:34)
- July shows higher number of lightining deals than June (26651 and 16169, respectively).
- Number of hourly sales normalized by number of deals is higher in July compared to June, as indicated by the monthly average vales and a right shifted distribution for July compared to June.



In [174]:
# Sales by date
sales_by_date = data_hourly[["hourly_sales_norm", "date"]].groupby(by="date").agg({"hourly_sales_norm": ["mean"]}).reset_index()
sales_by_date.columns = ["_".join(col) for col in sales_by_date.columns.values]
sales_by_date.rename(columns={"date_": "date"}, inplace=True)

sales_by_date =  sales_by_date.merge(right=data_hourly[["datetime_dayname", "date"]].drop_duplicates(), on="date", how="left")

with pd.option_context("display.max_rows", 62):
    display(sales_by_date)

,date,hourly_sales_norm_mean,datetime_dayname
0,2021-06-01,0.594005,Tuesday
1,2021-06-02,0.591541,Wednesday
2,2021-06-03,0.878512,Thursday
3,2021-06-04,0.561268,Friday
4,2021-06-05,0.616840,Saturday
5,2021-06-06,0.292608,Sunday
6,2021-06-07,0.574206,Monday
7,2021-06-08,0.838068,Tuesday
8,2021-06-09,0.623137,Wednesday
9,2021-06-10,0.730565,Thursday


In [175]:
fig = px.line(sales_by_date, x="date", y="hourly_sales_norm_mean", markers=True, hover_data=sales_by_date, width=1200, height=400)
fig.show()

##### Observation 9
A tendency for higher hourly sales is observed around the middle of the month (days 12-15). This is probably not necessarily a feature of this type of deals but more a normal trend of monthly sales, but to confirm this, we'd need to also have no-deals sales data.

In [186]:
# Sales by date of the week
sales_by_weekday = data_hourly[["hourly_sales_norm", "weekday", "datetime_dayname"]].groupby(by=["datetime_dayname", "weekday"]).agg({"hourly_sales_norm": ["mean"]}).reset_index()
sales_by_weekday.columns = ["_".join(col) for col in sales_by_weekday.columns.values]
sales_by_weekday.rename(columns={"datetime_dayname_": "datetime_dayname", "weekday_": "weekday"}, inplace=True)
sales_by_weekday = sales_by_weekday.sort_values("weekday")
display(sales_by_weekday)

,datetime_dayname,weekday,hourly_sales_norm_mean
1,Monday,0,0.927342
5,Tuesday,1,0.904948
6,Wednesday,2,0.895777
4,Thursday,3,0.866957
0,Friday,4,0.690631
2,Saturday,5,0.546303
3,Sunday,6,0.464941


In [187]:
fig = px.line(sales_by_weekday, x="datetime_dayname", y="hourly_sales_norm_mean", markers=True, hover_data=sales_by_weekday, width=1200, height=400)
fig.show()

In [139]:
# Sales by day of the week
sales_by_day = data[
    ["SOLD_QUANTITY", "SOLD_QUANTITY_HOURLY", "OFFER_START_DAYNAME"]
    ].groupby(
        by="OFFER_START_DAYNAME"
        ).agg(
            {
                "SOLD_QUANTITY_HOURLY": ["mean"]
            }
        ).reset_index()
sales_by_day.columns = ["_".join(col) for col in sales_by_day.columns.values]
sales_by_day.rename(columns={"OFFER_START_DAYNAME_": "OFFER_START_DAYNAME"}, inplace=True)
display(sales_by_day)

,OFFER_START_DAYNAME,SOLD_QUANTITY_HOURLY_mean
0,Friday,0.837410
1,Monday,1.198160
2,Saturday,0.667243
3,Sunday,0.612164
4,Thursday,0.955802
5,Tuesday,1.098217
6,Wednesday,1.078784


In [189]:
# Sales by hour of the day
sales_by_dayhour = data_hourly[["hourly_sales_norm", "datetime_hour"]].groupby(by=["datetime_hour"]).agg({"hourly_sales_norm": ["mean"]}).reset_index()
sales_by_dayhour.columns = ["_".join(col) for col in sales_by_dayhour.columns.values]
sales_by_dayhour.rename(columns={"datetime_hour_": "datetime_hour"}, inplace=True)
sales_by_dayhour = sales_by_dayhour.sort_values("datetime_hour")
display(sales_by_dayhour)

,datetime_hour,hourly_sales_norm_mean
0,0,0.450702
1,1,0.431961
2,2,0.342825
3,3,0.233229
4,4,0.264740
5,5,0.235420
6,6,0.348424
7,7,0.277280
8,8,0.277163
9,9,0.282625


In [191]:
fig = px.line(sales_by_dayhour, x="datetime_hour", y="hourly_sales_norm_mean", markers=True, hover_data=sales_by_dayhour, width=1200, height=400)
fig.show()

#### Distribution of sales among product categories

#### Effect of shipping payment type